# Correct NP precipitation following Yang

The notebook describes the algorithms to correct precipitation from North Pole drifting stations following the WMO approach described in Goodison et al (1998) and Yang (1999).

In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append('../source')

import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import readers.npsnow as npsnow
import utilities.correction as npcorr
from merge_npsnow_data import get_station_list, met_filename, get_precip, plot_station_met, merge_one_station

## Data
I use raw data from the NP drifting stations.  I have produced my own combined files that combine precipitation and meteorological observations.  These files are stored in /home/apbarret/Data/NPSNOW/my_combined_met

- TAIR - 2m air temperature
- RH - Relative humidity at 2m
- SLP - Sea level pressure
- WDIR - wind direction
- WSPD - wind speed m/s
- TOTCLD - total cloud cover
- LOWCLD - low cloud cover
- TSURF - surface temperature
- PRECIP - precipitation amount in mm  0 = Trace, when PTYPE != 0
- PTYPE - precipitation type
  - 1 - solid precipitation
  - 2 - mixed phase (rain/snow)
  - 3 - rain
- SDEPTH - snow depth

In [3]:
data_dir = '/home/apbarret/Data/NPSNOW/my_combined_met'
filelist = glob.glob(os.path.join(data_dir, 'npmet_??_combined.csv'))
filelist

['/home/apbarret/Data/NPSNOW/my_combined_met/npmet_29_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_31_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_26_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_24_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_25_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_22_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_28_combined.csv',
 '/home/apbarret/Data/NPSNOW/my_combined_met/npmet_30_combined.csv']

_Add description of data_

In [24]:
#df = npsnow.read_my_combined(filelist[0])
df = pd.concat([pd.read_csv(f, header=0, parse_dates=True) for f in filelist])
df.rename({'Unnamed: 0': 'Datetime'}, axis=1, inplace=True)
df.head()

,Datetime,Station_ID,TAIR,RH,SLP,WDIR,WSPD,TOTCLD,LOWCLD,TSURF,TMIN,TMAX,PRECIP,PTYPE,SDEPTH,Ug,Longitude,Latitude
0,1987-07-01 12:00:00,29.0,-0.4875,96.000,1023.4250,113.125,2.500,8.750000,7.000000,NaN,-1.4,0.2,0.0,3.0,0.0,1.917525,111.601616,80.514878
1,1987-07-02 12:00:00,29.0,0.1500,93.125,1026.3375,47.500,4.000,7.800000,3.571429,NaN,-0.7,1.1,0.3,3.0,0.0,3.068040,111.300398,80.530995
2,1987-07-03 12:00:00,29.0,0.2500,95.750,1020.6500,64.375,4.375,8.250000,0.750000,NaN,-1.2,1.3,0.4,3.0,0.0,3.355669,110.907852,80.564462
3,1987-07-04 12:00:00,29.0,0.0875,96.000,1018.4625,94.375,5.875,9.666667,9.666667,NaN,0.0,0.3,-9.9,-9.0,0.0,4.506184,110.529478,80.611184
4,1987-07-05 12:00:00,29.0,0.1625,99.000,1012.9250,145.625,3.000,10.000000,9.333333,NaN,-0.5,0.5,0.3,3.0,0.0,2.301030,110.258376,80.667667


In [36]:
type(df.Datetime.values[0])

str

In [29]:
df_table = df.pivot(index='Datetime', columns='Station_ID', values='PRECIP')

In [30]:
df_table.index

Index(['1973-10-01 12:00:00', '1973-10-02 12:00:00', '1973-10-03 12:00:00',
       '1973-10-04 12:00:00', '1973-10-05 12:00:00', '1973-10-06 12:00:00',
       '1973-10-07 12:00:00', '1973-10-08 12:00:00', '1973-10-09 12:00:00',
       '1973-10-10 12:00:00',
       ...
       '1991-03-22 12:00:00', '1991-03-23 12:00:00', '1991-03-24 12:00:00',
       '1991-03-25 12:00:00', '1991-03-26 12:00:00', '1991-03-27 12:00:00',
       '1991-03-28 12:00:00', '1991-03-29 12:00:00', '1991-03-30 12:00:00',
       '1991-03-31 12:00:00'],
      dtype='object', name='Datetime', length=6299)

## Merge met and precip data  - ***This is not required***
The met and precip files are resampled and merged into a single dataframe.  Daily maximum and minimum air temperatures are also added to the structure.  

sid = '21'
met = npsnow.read_met(met_filename(sid))
precDay = get_precip(sid) 
metDay = met.resample('D').mean()
metDay['TMIN'] = met['TAIR'].resample('D').min()
metDay['TMAX'] = met['TAIR'].resample('D').max()
merged_df = pd.concat([metDay,precDay], axis=1, sort=False)
merged_df = merged_df.rename({'amount': 'PRECIP', 'type': 'PTYPE'}, axis=1)
df = merged_df.drop('statid', axis=1)
df['Ug'] = df.apply(wind_at_gauge, axis=1)
df.head()

## Generate table to match Yang monthly tables - wind correction, trace correction and corrected precip left as Nan

In [7]:
dfMon = pd.DataFrame({
    'ND': df.Station_ID.resample('MS').count(),
    'Tmn': df.TAIR.resample('MS', label='left').mean(),
    'Ug': df.WSPD[df.WSPD < 6.].resample('MS', label='left').mean(),
    'DP': df.PRECIP[df.PRECIP > 0.].resample('MS').count(),
    'Dtc': df.PRECIP[df.PRECIP == 0.].resample('MS').count(),
    'Pg': df.PRECIP.resample('MS').sum()
})
dfMon

,ND,Tmn,Ug,DP,Dtc,Pg
1987-07-01,31,0.220161,4.171875,23,4,-19.6
1987-08-01,31,-1.032258,3.922619,24,2,-29.2
1987-09-01,30,-10.382917,4.345238,21,5,-18.4
1987-10-01,31,-16.390323,3.738636,23,5,-7.5
1987-11-01,30,-29.741667,3.925000,11,3,-151.0
1987-12-01,31,-32.711290,3.893750,6,1,-232.9
1988-01-01,31,-29.939919,4.875000,23,4,-16.6
1988-02-01,29,-29.355603,4.133333,19,4,-39.1
1988-03-01,31,-36.337500,3.554348,18,8,-35.5
1988-04-01,30,-28.451250,3.552885,11,5,-127.4


f = filelist[0]
df = pd.read_csv(f, header=0, index_col=0, sep=',', parse_dates=True)
df.head()

In [ ]:
plot_station_met(df, title=os.path.basename(f))

def cr_tretyakov_snow(ws, tmax, tmin):
    """Calculates the catch ratio of a Tretyakov rain gauge using WMO standard procedure for snow
    
    Methods for developing catch ratios are described in Goodison et al 1998.
    
    Arguments
    ---------
    ws - wind speed at height of gauge orifice in m/s
    tmax - maximum air temperature in degrees celsius
    tmin - minimum air temperature
    """
    return 103.11 - 8.67*ws + 0.3*tmax


def cr_tretyakov_mixed(ws, tmax, tmin):
    """Calculates the catch ratio of a Tretyakov rain gauge using WMO standard procedure for mixed precipitation
    
    Methods for developing catch ratios are described in Goodison et al 1998.
    
    Arguments
    ---------
    ws - wind speed at height of gauge orifice in m/s
    tmax - maximum air temperature in degrees celsius
    tmin - minimum air temperature in degrees celsius
    """
    return 96.99 - 4.46*ws + 0.88*tmax +0.22*tmin

def cr_tretyakov_rain(ws, tmax, tmin):
    """Returns catch ratio for Treyakov rain gauge using WMO standard procedure for rain.
    
    In Goodison et al 1998, no rain correction is available but they state that catch ratios for rain are 
    largely unaffected by wind.  So I take the average catch ratio of all sites from Table 4.4.1
    
    I drop catch ratios from Bismark and Harzgerode because these a lower by more than 5% than other locations."""
    return 91.7


def cr_tretyakov_dry(ws, tmax, tmin):
    return 100.

def catch_ratio(x):
    cr_func = {
        0: cr_tretyakov_dry,
        1: cr_tretyakov_snow,
        2: cr_tretyakov_mixed,
        3: cr_tretyakov_rain,
        }
    if x.WSPD < 6.:
        return 1./ (cr_func[x.PTYPE](x.WSPD, x.TMAX, x.TMIN)*0.01 )
    else:
        return 1.


def wind_at_gauge(x):
    """Reduces 10 m wind speed to wind at gauge height orifice"""
    H = 10.  # height of anenometer
    hg = 3.  # height of gauge orifice
    z0 = 0.01  # Roughness parameter of snow surface
    return x.WSPD * np.log10(hg/z0) / np.log10(H/z0)

Proof of concept for using average catch ratio for rain

In [ ]:
rain = pd.DataFrame({'wspd': [3.8, 2.7, 1.0, 2.5, 4.2, 3.3, 1.2, 1.6, 1.9, 3.9, 2.3],
                     'cr': [91.4, 92.0, 94.3, 86.6, 81.3, 71.6, 90.6, 88.2, 95.0, 97.4, 90.0]})
rain.drop([4,5]).mean()

In [ ]:
rain.plot(x='wspd', y='cr', marker='+', ls='')

Set Trace days to 0.1 mm

In [ ]:
df['PTRACE'] = 0.0
df.loc[(df['PTYPE'] > 0) & (df['PRECIP'] == 0.), 'PTRACE'] = 0.1
df['Ug'] = df.apply(wind_at_gauge, axis=1)
df['Ug'].where(df['Ug'] <= 6.).resample('M').mean()

In [ ]:
df['CR'] = df.apply(catch_ratio, axis=1)
df['PWIND'] = df['PRECIP'] * df['CR']
df.head()

In [ ]:
df['PCORR'] = df.PRECIP + df.PTRACE + df.PWIND
df.head()

In [ ]:
precMon = df[['PRECIP', 'PTRACE', 'PWIND', 'PCORR']].resample('M').sum()
precMon['TAIR'] = df['TAIR'].resample('M').mean()
precMon['WSPD'] = df['WSPD'].resample('M').mean()
precMon['TAIR_yang'] = yangMon['Tmn'].values
precMon['WSPD_yang'] = yangMon['Ug'].values
precMon

In [ ]:
sub = df[(df.index.year == 1972) & (df.index.month == 5)]
(sub.PTRACE * sub.CR).sum()

In [ ]:
yang_diri = '/home/apbarret/Data/NPSNOW/yang_precip'
yangMon = npsnow.read_yang_updated(os.path.join(yang_diri, f'yang_np_precip_updated_coords_{sid}.csv'))
yangMon.index = yangMon.Date
yangMon